<a href="https://colab.research.google.com/github/seoyeon0905/mimic-readmission-analysis/blob/main/notebooks/06_readmission_model_extension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 04 -> "이 정도로는 안 된다"
- 05 -> "그래도 이런 패턴은 있다"
- 06 -> "그럼 뭘 더 넣으면 좋아질까?"

# 06_readmission_model_extension.ipynb의 목표
- baseline(04) 대비 의미 있는 성능 개선
- features 확장 효과 검증
- 해석 가능한 수준에서 모델 비교

## 06에서 추가하면 좋은 Feature 레벨(우선순위)
### Level1: 입원 정보 확장
이미 df에 있는 것들
- admission_type
- admission_location
- discharge_location
- insurance
- hospital_expire_flag(careful)
### -> 05에서 봤던 risk factor를 모델에 넣는 단계

---

### Level2: 이진/범주 처리
- One-hot encoding
- 결측값 처리 전략 명시

---

### Level3: 모델 2~3개만 비교
- Logistic Regression(확장 버전)
- Tree-based(RandomForest or XGBoost 중 하나)

## 06. Extended Readmission Prediction Model

### Objective
To evaluate whether incorporating additional admission-related features
improves predictive performance compared to the baseline model.

---

## 06. 확장된 재입원 예측 모델

### 목적
본 노트북에서는 baseline 모델에서 사용한 제한적인 변수 구성에
입원 유형, 입원 경로, 퇴원 경로, 보험 유형과 같은
추가적인 입원 관련 변수를 포함함으로써,
30일 이내 ICU 재입원 예측 성능이 얼마나 개선되는지를 평가한다.

# Feature set
- Baseline set: age, icu_los
- Extended set: age, icu_los, admission_type,
  admission_location, discharge_location, insurance

# 전처리
- 범주형 변수에 대해 one-hot encoding 수행
- 결측값은 별도 처리 없이 기존 데이터 기준 유지
- Train/test split은 baseline 모델과 동일한 random_state 사용

# 모델
- Logistic Regression (extended)
- RandomForestClassifier (비선형 관계 비교)

# 평가 지표
- Recall
- PR-AUC
- AUROC
- Confusion Matrix

# 비교 및 해석
- Baseline 대비 성능 개선 여부
- 선형 모델과 트리 모델 간 차이
- 어떤 feature 유형이 재입원 예측에 기여하는지 해석